In [ ]:
from google.cloud import bigquery
import pandas as pd

PROJECT_ID = "book-project-479914"   # <-- your project

client = bigquery.Client(project=PROJECT_ID)

df = client.query("""
    SELECT *
    FROM `book-project-479914.harmonized_data.maybe_final_concat`
""").to_dataframe()


In [ ]:
!pip install rapidfuzz


In [ ]:
import pandas as pd
import numpy as np
import re
from rapidfuzz import fuzz, process
from tqdm import tqdm

# ----------------------------------------------------
# 0. CLEAN NUMERIC / BOOLEAN COLUMNS
# ----------------------------------------------------
for col in ['awards', 'bestseller', 'classic']:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# ----------------------------------------------------
# 1. CLEAN TITLE AND AUTHOR
# ----------------------------------------------------
def normalize_text(s):
    if pd.isna(s):
        return ""
    s = s.lower()
    s = re.sub(r'[^a-z0-9 ]+', '', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

df['title_clean'] = df['title'].apply(normalize_text)
df['author_clean'] = df['author'].apply(normalize_text)

# ----------------------------------------------------
# 2. BLOCKING: reduce comparisons
# ----------------------------------------------------
df['block'] = df['title_clean'].str[:5] + df['author_clean'].str[:3]

# ----------------------------------------------------
# 3. FUZZY CLUSTERING
# ----------------------------------------------------
cluster_id = 0
cluster_map = {}  # row index -> cluster id
assigned = set()

for block_val, group in tqdm(df.groupby('block'), desc="Processing blocks"):
    indices = group.index.tolist()
    titles = list(group['title_clean'])
    authors = list(group['author_clean'])
    
    while indices:
        idx0 = indices.pop(0)
        title0 = titles[group.index.get_loc(idx0)]
        author0 = authors[group.index.get_loc(idx0)]

        candidates = [i for i in indices if i not in assigned]
        if not candidates:
            cluster_map[idx0] = cluster_id
            cluster_id += 1
            continue

        candidate_titles = [df.at[i, 'title_clean'] for i in candidates]
        candidate_authors = [df.at[i, 'author_clean'] for i in candidates]

        title_scores = process.cdist([title0], candidate_titles, scorer=fuzz.token_sort_ratio)[0]
        author_scores = process.cdist([author0], candidate_authors, scorer=fuzz.token_sort_ratio)[0]

        matched_indices = [candidates[i] for i, (ts, as_) in enumerate(zip(title_scores, author_scores)) if ts >= 75 and as_ >= 75]

        # assign cluster
        cluster_map[idx0] = cluster_id
        for m in matched_indices:
            cluster_map[m] = cluster_id
            assigned.add(m)
            indices.remove(m)

        assigned.add(idx0)
        cluster_id += 1

df['cluster_id'] = df.index.map(cluster_map)

# ----------------------------------------------------
# 4. AGGREGATE CLUSTERS (VECTORIZE WITH AGG)
# ----------------------------------------------------
agg_funcs = {
    'source': pd.Series.nunique,
    'title': 'first',
    'author': 'first',
    'review': 'mean',
    'reviews_count': 'sum',
    'published_year': 'min',
    'price_eur': 'mean',
    'length': lambda x: x.value_counts().idxmax() if not x.value_counts().empty else None,
    'awards': 'max',
    'bestseller': 'max',
    'classic': 'max',
    'book_series': lambda x: 1 if x.notna().any() else 0,
    'genre': lambda x: ", ".join(x.dropna().unique()) if x.notna().any() else None,
    'years_on_bestsellers_list': 'sum'
}

df_final = df.groupby('cluster_id').agg(agg_funcs).reset_index()
df_final['price_eur'] = df_final['price_eur'].round(2)

# ----------------------------------------------------
# 5. FINAL OUTPUT
# ----------------------------------------------------
print(df_final.head())


In [ ]:
df_final.to_csv('deduplicated.csv')